In [1]:
import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import kendalltau

In [2]:
model_summary = FastText.load("models-4.1(a)/model_summary.bin")


In [11]:
data = pd.read_csv("data_ner.csv", sep=";")
headlines = data["ner_tokens"].astype(str).tolist()

In [12]:
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok[0]]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    head_vec.append(sum1)

/home/prachi/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [14]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = list(np.log2(np.array(data["Like Count"][:feat_len].tolist())+1))
test = head_vec[feat_len:]
res_test = list(np.log2(np.array(data["Like Count"][feat_len:].tolist())+1))
print(len(train), len(res_train))

7618 7618


In [15]:
model_svm = svmutil.svm_train(res_train, train,'-s 4 -t 2 -c 2')

In [16]:
y_reg = svmutil.svm_predict(res_test, test, model_svm)

Mean squared error = 0.847386 (regression)
Squared correlation coefficient = 6.25155e-05 (regression)


In [17]:
r2 = r2_score(res_test,y_reg[0])
r2

-0.013664958802295235

In [18]:
mae = mean_absolute_error(res_test, y_reg[0])
mae

0.7067043802231711

In [19]:
kt = kendalltau(res_test, y_reg[0])
kt

KendalltauResult(correlation=0.015408975797999769, pvalue=0.33507961991576574)

In [20]:
y_full = svmutil.svm_predict(res_test+res_train, test+train, model_svm)

Mean squared error = 0.933864 (regression)
Squared correlation coefficient = 0.00189295 (regression)


In [21]:
r2 = r2_score(res_test+res_train,y_full[0])
r2

-0.015148913883568982

In [22]:
mae = mean_absolute_error(res_test+res_train, y_full[0])
mae

0.7356078011313563

In [23]:
kt = kendalltau(res_test+res_train, y_full[0])
kt

KendalltauResult(correlation=0.02898987291307224, pvalue=4.7710098448617614e-05)

In [24]:
y_tr = svmutil.svm_predict(res_train, train, model_svm)

Mean squared error = 0.955489 (regression)
Squared correlation coefficient = 0.00268669 (regression)


In [25]:
r2 = r2_score(res_train,y_tr[0])
r2

-0.015552189748904599

In [26]:
mae = mean_absolute_error(res_train, y_tr[0])
mae

0.7428355534062439

In [27]:
kt = kendalltau(res_train, y_tr[0])
kt

KendalltauResult(correlation=0.032222363614037915, pvalue=5.236677514177423e-05)

In [28]:
model_svm_reg = svmutil.svm_train(res_train+res_test, train+test,'-s 4 -v 10 -t 2 -c 2')

Cross Validation Mean squared error = 0.934371
Cross Validation Squared correlation coefficient = 0.000779518


In [29]:
svmutil.svm_save_model("model_b_reg.bin",model_svm)